In [ ]:
from bs4 import BeautifulSoup   
import requests
from selenium import webdriver   
import time

#Go to http://myhttpheader.com/ to get 'User-Agent' + 'Accept-Language'
header = {
    'User-Agent': '',
    'Accept-Language': ''
}
#Beautiful Soup Documentation - https://www.crummy.com/software/BeautifulSoup/bs4/doc/
#requests.get() Syntax - requests.get(url, params={key: value}, args)
#note - args in this instance - headers = header
response = requests.get(
    'https://www.zillow.com/south-loop-chicago-il/condos/1-_beds/1.0-_baths/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22South%20Loop%2C%20Chicago%2C%20IL%22%2C%22mapBounds%22%3A%7B%22west%22%3A-87.65504208752441%2C%22east%22%3A-87.59581891247558%2C%22south%22%3A41.836086454828106%2C%22north%22%3A41.87789522196403%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A269607%2C%22regionType%22%3A8%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22price%22%3A%7B%22min%22%3A200000%2C%22max%22%3A250000%7D%2C%22mp%22%3A%7B%22min%22%3A708%2C%22max%22%3A885%7D%2C%22beds%22%3A%7B%22min%22%3A1%7D%2C%22baths%22%3A%7B%22min%22%3A1%7D%2C%22sort%22%3A%7B%22value%22%3A%22priced%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sf%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A14%7D', 
    headers = header)
data = response.text
soup = BeautifulSoup(data, 'html.parser')

#Get Property Links, Addresses and Prices - form the bases for the 3 questions on the Google Form that will be created subsequently
#Property Links
#note - one common task is extracting all the URLs found within a page’s <a> tags - href
property_links_elements = soup.select('.list-card-top a')
#NOT WORKING - property_links = [property_links.append(link[href]) for link in property_links_elements]

property_links = []
for link in property_links_elements:
    href = link['href']
    if 'http' not in href:
        property_links.append(f'https://www.zillow.com{href}')
    else:
        property_links.append(href)

property_addresses_elements = soup.select('.list-card-info address')
property_addresses = [address.get_text().split('|')[-1] for address in property_addresses_elements]

#NOT WORKING - property_prices_elements = soup.select('.list-card-heading price')
#NOT WORKING - property_prices = [property_prices.append(price) for price in property_prices_elements]

executable_path = '/Users/sabiahamid69/Desktop/Zain Adhami Files/Development/chromedriver'
driver = webdriver.Chrome(executable_path)

#note - apartment abbreviation - apt
for apt in range(len(property_links)):
    #create a google form - https://www.google.com/forms/about/
    driver.get('')
    time.sleep(1)
    #Selenium Documentation - https://selenium-python.readthedocs.io/locating-elements.html
    #note - find_element_by_xpath
    #What's the address of the property? XPath - type = 'text' + right_click(Copy XPath)
    property_address = driver.find_element_by_xpath('//*[@id="mG61Hd"]/div[2]/div/div[2]/div[1]/div/div/div[2]/div/div[1]/div/div[1]/input')
    #What's the price of the property? XPath - type = 'text' + right_click(Copy XPath)
    #property_price = driver.find_element_by_xpath('//*[@id="mG61Hd"]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div[1]/div/div[1]/input')
    #What's the link to the property? XPath - type = 'text' + right_click(Copy XPath)
    property_link = driver.find_element_by_xpath('//*[@id="mG61Hd"]/div[2]/div/div[2]/div[3]/div/div/div[2]/div/div[1]/div/div[1]/input')
    #Submit Button XPath - role = 'button' + right_click(Copy XPath)
    submit_button = driver.find_element_by_xpath('//*[@id="mG61Hd"]/div[2]/div/div[3]/div[1]/div/div')
    
    property_address.send_keys(property_addresses[apt])
    time.sleep(1)
    #NOT WORKING - property_price.send_keys(property_prices[apt])   
    property_link.send_keys(property_links[apt])
    time.sleep(1)
    
    submit_button.click()     